In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from glob import glob

from torchtext import vocab, data
from fastai.learner import *
from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import spacy
from spacy.symbols import ORTH
import dill as pickle



    Only loading the 'en' tokenizer.



In [3]:
#Process dataset - create dataset with 

PATH='/home/ubuntu/course-shortcut/competitions/lyrics_generator/'
TRN = "/home/ubuntu/course-shortcut/competitions/lyrics_generator/all/trn/"
VAL = "/home/ubuntu/course-shortcut/competitions/lyrics_generator/all/val/"

In [9]:
paths = glob(f'{TRN}/*.txt')
    

# text_field = data.Field(lower=True, tokenize=my_spacy_tok)
my_tok = spacy.load('en')
def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

text = []
text_field = data.Field(lower=True, tokenize=my_spacy_tok)
for p in paths:
    for line in open(p): text += text_field.preprocess(line)
    text.append('<eos>')



    Only loading the 'en' tokenizer.



In [10]:
paths = glob(f'{VAL}/*.txt')

# text_field = data.Field(lower=True, tokenize=my_spacy_tok)
my_tok = spacy.load('en')
def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

val_text = []
text_field = data.Field(lower=True, tokenize=my_spacy_tok)
for p in paths:
    for line in open(p): val_text += text_field.preprocess(line)
    val_text.append('<eos>')



    Only loading the 'en' tokenizer.



In [4]:
#Save the word list
trn_tokens = '/home/ubuntu/tokenized_words.pkl'
val_tokens = '/home/ubuntu/val_tokenized_words.pkl'
# pickle.dump(val_text, open('/home/ubuntu/val_tokenized_words.pkl','wb'))
# pickle.dump(text, open('/home/ubuntu/tokenized_words.pkl','wb'))
my_tok = spacy.load('en')
def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]
text_field = data.Field(lower=True, tokenize=my_spacy_tok)

text = pickle.load(open(trn_tokens, 'rb'))
val_text = pickle.load(open(val_tokens, 'rb'))



    Only loading the 'en' tokenizer.



In [10]:
# file.writelines
# file = open("/home/ubuntu/course-shortcut/competitions/lyrics_generator/all/trn.txt", 'w')

# for p in paths:
#     file.write(open(p).read())
#     file.write('<eos>')

# file.close()

In [5]:
def createDataSet(text_field, text):
    fields = [('text', text_field)]
    examples = [data.Example.fromlist([text], fields)]
    return data.Dataset(examples, fields)

In [6]:
trn_ds = createDataSet(text_field, text)

In [7]:
val_ds = createDataSet(text_field, val_text)

In [8]:
text_field.build_vocab(trn_ds, min_freq=10)

In [9]:
len(text_field.vocab)

16709

In [16]:
text_field.vocab.itos[:12]

['<unk>', '<pad>', ',', 'i', 'the', 'you', 'a', 'it', 'and', 'to', 'my', 'me']

In [10]:
em_sz = 500nh = 500nl = 3bs=32
bptt=300 #backpropogate through time: number of words it'll remember

FILES = dict(train=TRN, validation=VAL, test=VAL)

md = LanguageModelData.from_text_files(PATH, text_field, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [11]:
optimization_function = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(optimization_function, em_sz, nh, nl,
dropout=0.3, dropouth=0.3, dropouti=0.3, dropoute=0.3, wdrop=0.2)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)

In [19]:
learner.clip=0.3
learner.fit(0.01, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       5.41376  5.09887]                                



In [21]:
learner.save("kam-test1")

In [20]:
from random import randint
def proc_str(s): return text_field.preprocess(text_field.tokenize(s))
def num_str(s): return text_field.numericalize([proc_str(s)])
model=learner.model

def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')    

    for i in range(l):
        pred = res
        # get top 5 predictions
#         top_pred = pred.topk(5) #.exp() #/ pred[-1].exp().sum()    
        top_prob = 3
        rand = randint(0, top_prob - 1)
        _, top_indices = pred[-1].topk(top_prob)
        # Randomly sample one word
        n = top_indices.data[rand]
        word = text_field.vocab.itos[n]
        print(word, end=' ')
        if word=='<eos>': break
        if word==',': print("\n")
        res,*_ = m(num_str(word))

    m[0].bs=bs

In [24]:
sample_model(model, "One")

...of the <unk> and i do it like the same <unk> <unk> - a <unk> - <unk> , 

i 'm the <unk> , 

<unk> , 

<unk> - <unk> , 

<unk> , 

<unk> <unk> e <unk> <unk> , 

e <unk> <unk> e e e e , 

a a a a , 



In [37]:
learner.fit(0.05, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))


  1%|▏         | 9/631 [00:07<08:05,  1.28it/s, loss=7.91]


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/envs/latest-fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda2/envs/latest-fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda2/envs/latest-fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




  6%|▌         | 38/631 [00:28<07:20,  1.35it/s, loss=14.1]


KeyboardInterrupt: 

In [25]:
learner.fit(0.005, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       5.16789  4.82935]                                



In [26]:
learner.save("kam-test2")

In [13]:
learner.load('kam-test2')

In [14]:
learner.fit(0.001, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       5.18342  4.74189]                                
[ 1.       5.15557  4.69803]                                



In [15]:
learner.save("kam-test3")

In [37]:
learner.load("kam-test3")

In [38]:
learner.fit(0.0001, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       5.11138  4.68021]                                
[ 1.       5.10083  4.67461]                                



In [39]:
learner.save("kam-test4")

In [21]:
def sample_model2(model, s, num_words=50, random_level=4):
    words = s
    t = num_str(s)
    model[0].bs=1
    model.eval()
    model.reset()
    res,*_ = model(t)
    print('...', end='')

    for i in range(num_words):
        pred = res
        # get top number of predictions
        rand = randint(0, random_level - 1)
        _, top_indices = pred[-1].topk(random_level)
        # Randomly sample one word
        n = top_indices.data[rand]
        word = text_field.vocab.itos[n]
        print(word, end=' ')
        if word=='<eos>': break
        if word=='<unk>': continue
        if word==',': print("\n")

        words += " " + word
        num_words_to_remember = 200
        if len(words.split(" ")) > num_words_to_remember:
            words = "".join(words.split(" ")[-num_words_to_remember:])
        res,*_ = model(num_str(words))

    model[0].bs=bs

In [44]:
sample_model2(model, "I love you", num_words=200, random_level=4)

...when it comes and i do nt wanna be the only thing i love i know , 

i do i do n't need it but it feels good , 

you 're the only thing you want i love you when it goes i 'm on it i love the world that i love it i do it , 

oh , 

i love the time it feels good i 'm so good , 

i wanna go to the sky and i do n't want no more , 

you do it , 

i love you when i 'm in it when you get the money , 

yeah , 

yeah you know it i do , 

it 's all about the good time and it do i need to do it i want the way it 's the only way you got it you know it i 'm the way it 's the one that you want i know it i know , 

you 're so good i can see that you 're in my way , 

it ai the way you want to do , 

you can get your time on my life and if it ai all over and , 

<unk> 

In [32]:
sample_model2(model, "Shit", num_words=200)

..., 

we all in love , 

fuck we all we love , 

yeah yeah yeah i 'm all the bitches yeah yeah we got it on all the bitches i 'm the shit yeah yeah we got ta get the fuck up yeah we be on that , 

all these bottles we got ta get it , 

all night yeah yeah , 

i know that you 're all i need , 

you 're my girl i 'm the shit i want to make it , 

yeah yeah yeah i know , 

all my money i want you , 

i know you know i 'm on that shit yeah , 

you do that , 

all that you got , 

i got it , 

you , 

yeah , 

yeah you do n't know , 

i know you know it i 'm a make a dance , 

oh oh yeah i do what i want i like , 

you want it i do it for me , 

yeah i want you i need , 

i want you , 

yeah yeah i love my way <eos> 

In [12]:
learner.load("kam-test4")

In [13]:
learner.fit(0.00001, 2)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[ 0.       5.10981  4.67423]                                
[ 1.       5.11601  4.67587]                                



In [14]:
learner.fit(3e-3, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       5.19002  4.69439]                                



In [15]:
learner.fit(3e-4, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       5.083    4.63809]                                



In [16]:
learner.save('kam-test5')

In [17]:
learner.fit(3e-4, 3, wds=1e-6, cycle_len=2, cycle_mult=3)

HBox(children=(IntProgress(value=0, description='Epoch', max=26), HTML(value='')))

[ 0.       5.08655  4.62619]                                
[ 1.       5.07559  4.62677]                                
[ 2.       5.08182  4.60987]                                
[ 3.       5.06883  4.60078]                                
[ 4.       5.05049  4.58923]                                
[ 5.       5.04977  4.5873 ]                                
[ 6.       5.06919  4.58579]                                
[ 7.       5.04963  4.58734]                                
[ 8.       5.04825  4.58122]                                
[ 9.       5.04068  4.56802]                                
[ 10.        5.0298    4.56243]                             
[ 11.        5.01615   4.55334]                             
[ 12.        5.00283   4.54655]                             
[ 13.        4.99498   4.54109]                             
[ 14.        4.98789   4.53393]                             
[ 15.        4.99476   4.52855]                             
[ 16.        4.98982   4

KeyboardInterrupt: 

In [18]:
learner.save('kam-test6')

In [19]:
learner.fit(3e-3, 1, wds=1e-6)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))


  2%|▏         | 14/631 [00:09<07:16,  1.41it/s, loss=5.02]


Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/envs/latest-fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda2/envs/latest-fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ubuntu/anaconda2/envs/latest-fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




 10%|▉         | 61/631 [00:44<06:59,  1.36it/s, loss=5.04]


KeyboardInterrupt: 

In [25]:
sample_model2(learner.model, "take it back to the crib")

...and i got it for the money , 

i do for my money , 

yeah i do for it all my bitches , 

all the money i love 'em , 

yeah , 

fuck all my hoes , 

all these hoes bitches love 'em fuck niggas fuck me bitches i love 